# Python Dataclass

<!-- 파이썬에서는 동적타이핑언어로 데이터 타입에 대한 선언이 없어도 충분히 사용이 가능하다. 하지만 프로그램이 확장되고 데이터가 많아질 수록 타입에 대한 정의가 없다면 가독성이 떨어지고 개발자간 의사소통이 어려워진다. 결국 다수의 버그를 발생시키는 상황을 초래할 수 있다. -->

> Dataclass에 정말 상세하고 쉽게 설명해주신 [달레](https://www.daleseo.com/python-dataclasses/)님의 블로그를 참고하여 코드를 각색하였습니다.

## Dataclass 이전의 데이터 정의

In [1]:
from datetime import date

class Car:
    def __init__(
        self, id: int, model: str, create_date: date
    ) -> None:
        self.id = id
        self.model = model
        self.create_date = create_date

k5 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5

dataclass 이전의 데이터를 정의하기 위한 클래스는 `__init__()` 메서드로 인스턴스 변수를 생성시키는데, `__init__()` 메서드에 정의를 하는 것에서 입력, 정의 등에서 여러번 같은 변수명을 입력해야하는 번거로움이 있다.

그리고 인스턴스를 생성하였을 때에도 인스턴스 객체만을 출력하면 생성된 메모리의 위치를 알려줄 뿐이고, 실제 데이터를 감추고 있다.

이때 `__repr__()`메서드를 추가하면 인스턴스를 출력할 때, 필드값이 함께 출력되도록 정의할 수 있다.

In [2]:
from datetime import date

class Car:
    def __init__(
        self, id: int, model: str, create_date: date
    ) -> None:
        self.id = id
        self.model = model
        self.create_date = create_date

    def __repr__(self):
            return (
                self.__class__.__qualname__ + f"(id={self.id!r}, model={self.model!r}, "
                f"create_date={self.create_date!r})"
            )

k5 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))

In [3]:
# __repr__() !?

다음은 `__repr__()` 구현한 클래스에서 인스턴스를 생성할 때, 동일한 필드를 입력했을 때 동등한지 보겠다.

In [4]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
print(k5_1, k5_2, sep="\n")
print(k5_1 == k5_2)

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
False


동일한 필드가 나오지만, 서로 다른 인스턴스로 취급되는 것을 확인할 수 있다.

이때 동일한 필드를 입력할 때 동일한 인스턴스로 취급하려면, `__eq__()` 메서드를 추가해야한다.

In [5]:
from datetime import date

class Car:
    def __init__(
        self, id: int, model: str, create_date: date
    ) -> None:
        self.id = id
        self.model = model
        self.create_date = create_date

    def __repr__(self):
            return (
                self.__class__.__qualname__ + f"(id={self.id!r}, model={self.model!r}, "
                f"create_date={self.create_date!r})"
            )
# __eq__ 도 나중에
    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.model, self.create_date) == (
                other.id,
                other.model,
                other.create_date,
            )
        return NotImplemented

In [6]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
print(k5_1, k5_2, sep="\n")
print(k5_1 == k5_2)

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
True


## 데이터 클래스로 작성하기

다음은 위에서 작업하였던 코드들을 dataclass로 구현한다면 어떻게 될지 확인해보도록 하겠다.

In [7]:
from dataclasses import dataclass
from datetime import date


@dataclass
class Car:
    id: int
    model: str
    create_date: date

In [8]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
print(k5_1, k5_2, sep="\n")
print(k5_1 == k5_2)

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
True


위에서 구현했던 `__repr__()`, `__eq__()` 등이 dataclass로 랩핑해줌으로써 모두 구현이 되었다. 매우 간단하게 작성이 되었다.

### dataclass frozen 옵션?

위에서 작성된 데이터클래스를 immutable 해지기 위해 frozen 옵션을 사용할 수 있다.

기본 디포트로 False로 세팅되어 있기 때문에 `@dataclass(frozen=True)`와 같이 사용할 수 있다. 

frozen 옵션 `True`와 `False`로 비교해보자.

In [9]:
from dataclasses import dataclass
from datetime import date


@dataclass(frozen=False)
class Car:
    id: int
    model: str
    create_date: date


@dataclass(frozen=True)
class CarFrozen:
    id: int
    model: str
    create_date: date


In [10]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_1_frozen = CarFrozen(id=1, model="K5", create_date=date(2022, 2, 17))
k5_1, k5_1_frozen

(Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17)),
 CarFrozen(id=1, model='K5', create_date=datetime.date(2022, 2, 17)))

In [11]:
k5_1.model = "K3"
k5_1

Car(id=1, model='K3', create_date=datetime.date(2022, 2, 17))

k5_1의 model을 K3로 변경해 사용하였다.

In [12]:
k5_1_frozen.model = "K3"
k5_1_frozen

FrozenInstanceError: cannot assign to field 'model'

위와 같이 Frozen True일때 에러가 발생하게 된다.

### 대소비교 및 정렬 (order 옵션)

데이터 클래스를 사용한다면 비교연산을 통한 비교가 가능해진다. 따라서 정렬까지도 가능해진다.

In [20]:
@dataclass(order=True)
class Car:
    id: int
    model: str
    create_date: date

k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=2, model="K5", create_date=date(2022, 2, 17))
(k5_1 > k5_2), (k5_1 < k5_2)

(False, True)

여기서 궁금한 점, 여러 필드 중 어떤 필드를 통해 정렬을 하는 것일까?

In [30]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
(k5_1 == k5_2), (k5_1 < k5_2), (k5_1 > k5_2)

(True, False, False)

In [34]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 18))
(k5_1 > k5_2), (k5_1 < k5_2)

(False, True)

In [35]:
k5_1 = Car(id=2, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 18))
(k5_1 > k5_2), (k5_1 < k5_2)

(True, False)

In [38]:
k5_1 = Car(id=2, model="A5", create_date=date(2022, 2, 18))
k5_2 = Car(id=2, model="K5", create_date=date(2022, 2, 17))
(k5_1 > k5_2), (k5_1 < k5_2)

(False, True)

위의 세가지 경우에서 볼 수 있듯이 필드를 선언해준 가장 윗 순위부터 대소를 비교하는 것 같다.

In [41]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 15))
k5_2 = Car(id=2, model="K5", create_date=date(2022, 2, 16))
k5_3 = Car(id=3, model="K5", create_date=date(2022, 2, 13))
k5_4 = Car(id=4, model="K5", create_date=date(2022, 2, 18))
k5_5 = Car(id=5, model="K5", create_date=date(2022, 2, 19))

a = (k5_1, k5_2, k5_3, k5_4, k5_5)

sorted(a, reverse=True, key=lambda x: x.create_date)

[Car(id=5, model='K5', create_date=datetime.date(2022, 2, 19)),
 Car(id=4, model='K5', create_date=datetime.date(2022, 2, 18)),
 Car(id=2, model='K5', create_date=datetime.date(2022, 2, 16)),
 Car(id=1, model='K5', create_date=datetime.date(2022, 2, 15)),
 Car(id=3, model='K5', create_date=datetime.date(2022, 2, 13))]

정렬을 한다면 위와 같이 활용할 수 있겠다.

### set, dictionary에서 활용

데이터 클래스는 hashable 하지 않다. 

> 여기서 hashable은 hash함수에 인자로 들어갈 수 있는 객체를 의미한다. iterator가 iterable한 객체를 받는 것과 비슷한 의미인듯 하다. hash는 데이터를 다룰 때 쓰는 기법으로 key와 정수 value로 이루어져 검색과 저장을 매우 빠르게 할 수 있다. 이와 관련해서는 추후에 더 알아보도록 하자.

다시 돌아와서 데이터클래스는 hashable 하지 않기 때문에, set과 dictionary의 key로 사용되지 못한다.

In [44]:
{k5_1, k5_2}

TypeError: unhashable type: 'Car'

#### unsafe_hash 옵션
이럴 때 unsafe_hash 옵션을 True로 주면 hashable 하게 사용할 수 있다. 이때 hashable을 쓰는게 어떤 의미가 있냐라고 하면, set을 통해 중복을 제거 할 수 있게된다.

당연하게도 모든 필드에서 값이 동일해야 가능하다. 물론 중복은 이미 데이터를 정의할 때 스키마에서 validation check를 하는 것이 가장 좋겠다.

In [50]:
@dataclass(unsafe_hash=True)
class Car:
    id: int
    model: str
    create_date: date

k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 15))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 15))
k5_3 = Car(id=1, model="K5", create_date=date(2022, 2, 15))
k5_4 = Car(id=4, model="K5", create_date=date(2022, 2, 18))
k5_5 = Car(id=5, model="K5", create_date=date(2022, 2, 19))

{k5_1, k5_2, k5_3, k5_4, k5_5}

{Car(id=1, model='K5', create_date=datetime.date(2022, 2, 15)),
 Car(id=3, model='K5', create_date=datetime.date(2022, 2, 13)),
 Car(id=4, model='K5', create_date=datetime.date(2022, 2, 18)),
 Car(id=5, model='K5', create_date=datetime.date(2022, 2, 19))}

## 데이터 클래스 사용 시 주의사항

데이터 클래스를 사용할 때는 list과 같은 mutable 데이터를 어노테이션을 주면 에러가 발생한다.

필드의 디포트값은 인스턴스 간에 공유가 되어 디포트값 할당이 허용되지 않는다고 한다.

list를 데이터타입으로 줘야할 경우에는 데이터클래스의 `filed(default_factory=list)` 메서드를 활용할 수 있다.

In [53]:
from dataclasses import dataclass, field
from datetime import date
from typing import List

@dataclass(unsafe_hash=True)
class Car:
    id: int
    model: str
    create_date: date
    color_options: List[str] = field(default_factory=list)

k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 15), color_options=["red", "blue"])
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 15), color_options=["red", "blue", "black"])
k5_1, k5_2

(Car(id=1, model='K5', create_date=datetime.date(2022, 2, 15), color_options=['red', 'blue']),
 Car(id=1, model='K5', create_date=datetime.date(2022, 2, 15), color_options=['red', 'blue', 'black']))